In [ ]:
import h5py
import numpy as np
import pandas as pd
import json
import yaml
from ludwig import LudwigModel
import copy
import ray
from ludwig.utils.misc import merge_dict
import re

(pid=59547) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=59554) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=59555) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=59549) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=59553) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=59551) WARNING: Not monitoring node memory since `psutil` is not installed. Inst

In [93]:
pattern = "^{{.*}}$"
list_index_pattern = "^\[\d+\]$"


def get_keys_from_list(l, path=""):
    parameters = []
    
    for index, v in enumerate(l):
        if isinstance(l[index], str):
             if re.match(pattern, l[index], flags=0) is not None:
                parameters.append([path + "[" + index + "]", v])
                
        elif isinstance(l[index], dict):
            p = get_keys_from_dict(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)
        elif isinstance(l[index], list):
            p = get_keys_from_list(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)
                    
    return parameters
        

def get_keys_from_dict(dct, path=""):
    parameters = []
    for k, v in dct.items():
        if isinstance(dct[k], str):
            if re.match(pattern, dct[k], flags=0) is not None:
                parameters.append([path + k,dct[k]])
       
        elif isinstance(dct[k], dict):
            p = get_keys_from_dict(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)
            
        elif isinstance(dct[k], list):
            p = get_keys_from_list(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)
       
    return parameters

#updates parameters...doens't handle lists

def update_param(dct, path, value):
    if len(path) == 1:
        dct[path[0]] = value
    else:
        l = re.match(list_index_pattern, path[0], flags=0) 
        if l is not None and isinstance(dct, list):
            index = int(l.group(0)[1:-1])
            update_param(dct[index], path[1:], value)
        else:
             update_param(dct[path[0]], path[1:], value)    
        

def build_model(base_model, config):
    for k, v in config.items():
        p = k.split('->')
        update_param(base_model, p, v)
    return base_model




In [81]:
from ray.tune import grid_search 

def build_search_space(annotated_model):
    pattern = "^{{(.*)}}$"
    grid_search_space = {}
    keys = get_keys_from_dict(base_model_h)
    for a in keys:
        grid_search_space[a[0]]= grid_search(yaml.load(re.match(pattern, a[1], flags=0)[1]))
        
    return grid_search_space




In [82]:
#base should contain special markup so we know what parameters need to be updated.
#we don't need this anymore
def build_trial(base, config):
    
    combiner = base['combiner']
    training = base['training']
    
    
    combiner = merge_dict(combiner, {'num_fc_layers': config['num_fc_layers']})
    training = merge_dict(training, {'batch_size': config['batch_size']})
    
    new_model_def = {'input_features': base['input_features'], 
                 'output_features': base['output_features'], 
                 'combiner': combiner, 
                 'training': training}
    
    return new_model_def
   
    
    

In [91]:
data = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.hdf5'
metadata = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.json'

def train(base, config, reporter):
    print("******************")
    #print(base['input_features'])
    #print(config)
    new_model_def = build_model(base, config)
    print(new_model_def)
    
    model = LudwigModel(new_model_def)
    train_stats = model.train(data_hdf5=data, train_set_metadata_json=metadata)
    return reporter(mean_accuracy=train_stats['validation']['Survived']['accuracy'][-1], done=True)



    


In [94]:
from ray.tune import register_trainable, grid_search, run_experiments

ray.shutdown()
ray.init()
with open("titanic_full_h.yaml", 'r') as stream:
    annotated_model = yaml.load(stream)

'''grid_search_space = {
    'num_fc_layers': grid_search([1,2,3,4]),
    'batch_size': grid_search([4,16,32,64,128])
}'''

grid_search_space = build_search_space(annotated_model)
print(annotated_model)

register_trainable('train', lambda cfg, rptr: train(annotated_model, cfg, rptr))
run_experiments({'my_experiment': {
    'run': 'train',
    'stop': {'mean_accuracy': 0.9},
    'config': grid_search_space}
    })
                

2019-03-25 11:15:52,398	WARNING worker.py:1381 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-03-25 11:15:52,401	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-25_11-15-52_54401/logs.
2019-03-25 11:15:52,519	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:31928 to respond...
2019-03-25 11:15:52,651	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:12999 to respond...
2019-03-25 11:15:52,655	INFO services.py:761 -- Starting Redis shard with 3.44 GB max memory.
2019-03-25 11:15:52,695	INFO services.py:1449 -- Starting the Plasma object store with 5.15 GB memory using /tmp.
/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please re

{'combiner': {'fc_size': '{{[24,48,64]}}', 'num_fc_layers': 1, 'type': 'concat'}, 'input_features': [{'name': 'Pclass', 'type': 'category'}, {'name': 'Sex', 'type': 'category'}, {'name': 'Age', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'SibSp', 'type': 'numerical'}, {'name': 'Parch', 'type': 'numerical'}, {'name': 'Fare', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'Embarked', 'representation': '{{[sparse, dense]}}', 'type': 'category'}], 'output_features': [{'name': 'Survived', 'type': 'binary'}], 'training': {'batch_size': '{{[8,16,32,64]}}', 'bucketing_field': None, 'decay': False, 'decay_rate': 0.96, 'decay_steps': 10000, 'dropout_rate': 0.0, 'early_stop': 5, 'epochs': 100, 'gradient_clipping': None, 'increase_batch_size_on_plateau': 0, 'increase_batch_size_on_plateau_max': 512, 'increase_batch_size_on_plateau_patience': 5, 'increase_batch_size_on_plateau_rate': 2, 'learning_rate': 0.001, 'learning_rate_warmup_epochs':

(pid=62674) 2019-03-25 11:16:19.807307: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=62676) 2019-03-25 11:16:19.883708: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=62673) ******************
(pid=62673) [{'name': 'Pclass', 'type': 'category'}, {'name': 'Sex', 'type': 'category'}, {'name': 'Age', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'SibSp', 'type': 'numerical'}, {'name': 'Parch', 'type': 'numerical'}, {'name': 'Fare', 'type': 'numerical', 'missing_value_strategy': 'fill_with_mean'}, {'name': 'Embarked', 'representation': '{{[sparse, dense]}}', 'type': 'category'}]
(pid=62673) {'combiner->fc_size': 24, 'input_features->[6]->representation': 'sparse', 'training->batch_size': 8, 'training->optimizer->type': 'adam'}
(pid=62673) /Users/bm255022/

(pid=62676) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=62722) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=62722)   return f(*args, **kwds)
Result for train_2_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=adam:
  date: 2019-03-25_11-16-30
  done: true
  experiment_id: 2b54e414c5bf47229bae143e5b713174
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.8148148148148148
  node_ip: 10.0.9.192
  pid: 62679
  time_since_restore: 10.043805360794067
  time_this_iter_s: 10.043805360794067
  time_total_s: 10.043805360794067
  timestamp: 1553526990
  timesteps_since_restore: 0
  training_iteration

(pid=62674) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=62724) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=62724)   return f(*args, **kwds)
Result for train_1_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=adam:
  date: 2019-03-25_11-16-41
  done: true
  experiment_id: a08d51ab4d5b4d9b8b2539ba2a4e2ffb
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7901234567901234
  node_ip: 10.0.9.192
  pid: 62675
  time_since_restore: 15.190317153930664
  time_this_iter_s: 15.190317153930664
  time_total_s: 15.190317153930664
  timestamp: 1553527001
  timesteps_since_restore: 0
  training_iteration

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/bm255022/ray_results/my_experiment
Number of trials: 48 ({'TERMINATED': 24, 'ERROR': 23, 'RUNNING': 1})
ERROR trials:
 - train_24_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd:	ERROR, 1 failures: /Users/bm255022/ray_results/my_experiment/train_24_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd_2019-03-25_11-17-47qvo5dmea/error_2019-03-25_11-18-15.txt
 - train_25_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd:	ERROR, 1 failures: /Users/bm255022/ray_results/my_experiment/train_25_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd_

2019-03-25 11:19:21,889	ERROR trial_runner.py:426 -- Error processing event.
Traceback (most recent call last):
  File "/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 389, in _process_events
    result = self.trial_executor.fetch_result(trial)
  File "/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 252, in fetch_result
    result = ray.get(trial_future[0])
  File "/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ray/worker.py", line 2288, in get
    raise value
ray.exceptions.RayTaskError: ray_worker (pid=63010, host=MUSBM255022-163)
  File "/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ray/tune/trainable.py", line 150, in train
    result = self._train()
  File "/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 130, in _train
    result = self._status_re

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/bm255022/ray_results/my_experiment
Number of trials: 48 ({'TERMINATED': 24, 'ERROR': 24})
ERROR trials:
 - train_24_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd:	ERROR, 1 failures: /Users/bm255022/ray_results/my_experiment/train_24_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd_2019-03-25_11-17-47qvo5dmea/error_2019-03-25_11-18-15.txt
 - train_25_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd:	ERROR, 1 failures: /Users/bm255022/ray_results/my_experiment/train_25_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd_2019-03-25_11-

TuneError: ('Trials did not complete', [train_24_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd, train_25_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd, train_26_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=8,training->optimizer->type=sgd, train_27_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=8,training->optimizer->type=sgd, train_28_combiner->fc_size=48,input_features->[6]->representation=dense,training->batch_size=8,training->optimizer->type=sgd, train_29_combiner->fc_size=64,input_features->[6]->representation=dense,training->batch_size=8,training->optimizer->type=sgd, train_30_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=16,training->optimizer->type=sgd, train_31_combiner->fc_size=48,input_features->[6]->representation=sparse,

(pid=63010) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.


a more eleborate approach to specifying parameters to search over..

In [72]:
with open("titanic_full_h.yaml", 'r') as stream:
    base_model_h = yaml.load(stream)
base_model_h
        

/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


{'combiner': {'fc_size': '{{[24,48,64]}}',
  'num_fc_layers': 1,
  'type': 'concat'},
 'input_features': [{'name': 'Pclass', 'type': 'category'},
  {'name': 'Sex', 'type': 'category'},
  {'name': 'Age',
   'type': 'numerical',
   'missing_value_strategy': 'fill_with_mean'},
  {'name': 'SibSp', 'type': 'numerical'},
  {'name': 'Parch', 'type': 'numerical'},
  {'name': 'Fare',
   'type': 'numerical',
   'missing_value_strategy': 'fill_with_mean'},
  {'name': 'Embarked',
   'representation': '{{[sparse, dense]}}',
   'type': 'category'}],
 'output_features': [{'name': 'Survived', 'type': 'binary'}],
 'training': {'batch_size': '{{[8,16,32,64]}}',
  'bucketing_field': None,
  'decay': False,
  'decay_rate': 0.96,
  'decay_steps': 10000,
  'dropout_rate': 0.0,
  'early_stop': 5,
  'epochs': 100,
  'gradient_clipping': None,
  'increase_batch_size_on_plateau': 0,
  'increase_batch_size_on_plateau_max': 512,
  'increase_batch_size_on_plateau_patience': 5,
  'increase_batch_size_on_plateau_rat

In [73]:
keys = get_keys_from_dict(base_model_h)
keys

[['combiner->fc_size', '{{[24,48,64]}}'],
 ['input_features->[6]->representation', '{{[sparse, dense]}}'],
 ['training->batch_size', '{{[8,16,32,64]}}'],
 ['training->optimizer->type', '{{[adam,sgd]}}']]

In [59]:
config = {'combiner->fc_size': 48,
          'training->batch_size': 64,
          'training->optimizer->type': 'adam',
          'input_features->[6]->representation': 'dense'
         }

build_model(base_model_h, config)
base_model_h

{'combiner': {'fc_size': 48, 'num_fc_layers': 1, 'type': 'concat'},
 'input_features': [{'name': 'Pclass', 'type': 'category'},
  {'name': 'Sex', 'type': 'category'},
  {'name': 'Age',
   'type': 'numerical',
   'missing_value_strategy': 'fill_with_mean'},
  {'name': 'SibSp', 'type': 'numerical'},
  {'name': 'Parch', 'type': 'numerical'},
  {'name': 'Fare',
   'type': 'numerical',
   'missing_value_strategy': 'fill_with_mean'},
  {'name': 'Embarked', 'representation': 'dense', 'type': 'category'}],
 'output_features': [{'name': 'Survived', 'type': 'binary'}],
 'training': {'batch_size': 64,
  'bucketing_field': None,
  'decay': False,
  'decay_rate': 0.96,
  'decay_steps': 10000,
  'dropout_rate': 0.0,
  'early_stop': 5,
  'epochs': 100,
  'gradient_clipping': None,
  'increase_batch_size_on_plateau': 0,
  'increase_batch_size_on_plateau_max': 512,
  'increase_batch_size_on_plateau_patience': 5,
  'increase_batch_size_on_plateau_rate': 2,
  'learning_rate': 0.001,
  'learning_rate_warm

In [50]:
list_index_pattern = "^\[\d+\]$"
l = re.match(list_index_pattern, "[24]", flags=0)
int(l.group(0)[1:-1])

24

In [42]:
int(list(filter(str.isdigit, "[23]"))[0])

2

In [66]:
yaml.load("[a,b,c]")

/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


['a', 'b', 'c']